In [1]:
from google.colab import drive
drive.mount("/gdrive", force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
cd /gdrive/My\ Drive/Data Science/YBIGTA/2019-1_4_Conference/4_modeling

/gdrive/My Drive/Data Science/YBIGTA/2019-1_4_Conference/4_modeling


In [3]:
!ls

all_embbeding_df_v2.pickle  tfidf.pickle
all_for_all_sanghyun.ipynb  tfidf_w2v_cbow_avg.pickle
comparisons.ipynb	    tfidf_w2v_sg_avg.pickle
SVM.ipynb		    undersampling_lightGBM.ipynb
tfidf_ft_cbow_avg.pickle    Untitled0.ipynb
tfidf_ft_sg_avg.pickle	    zebal_version2.pickle


In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals
import tensorflow as tf
from tensorflow import keras
import pickle

import numpy as np

print(tf.__version__)

1.14.0-rc1


In [0]:
import pandas as pd
import numpy as np

In [0]:
with open('zebal_version2.pickle', 'rb') as f:
    df = pickle.load(f)

In [0]:
df = df[df['board']!='가입 인사 (등업 필수)']
df = df.reset_index()

In [0]:
df['L1'] = df['type'].apply(lambda x: x[0].upper())
df['L2'] = df['type'].apply(lambda x: x[1].upper())
df['L3'] = df['type'].apply(lambda x: x[2].upper())
df['L4'] = df['type'].apply(lambda x: x[3].upper())

In [0]:
with open('tfidf.pickle', 'rb') as f:
    tfidf = pickle.load(f)
with open('tfidf_ft_cbow_avg.pickle', 'rb') as f:
    tfidf_ft_cbow = pickle.load(f)
with open('tfidf_ft_sg_avg.pickle', 'rb') as f:
    tfidf_ft_sg = pickle.load(f)
with open('tfidf_w2v_cbow_avg.pickle', 'rb') as f:
    tfidf_w2v_cbow = pickle.load(f)
with open('tfidf_w2v_sg_avg.pickle', 'rb') as f:
    tfidf_w2v_sg = pickle.load(f)

In [0]:
with open('all_embbeding_df_v2.pickle', 'rb') as f:
    em = pickle.load(f)

In [0]:
def make_matrix(ndf):
    target=[]
    for i in ndf:
        target.append(i)
    return pd.DataFrame(target)

In [0]:
d2v_dvow = make_matrix(em['doc2vec_PV_DBOW'])
d2v_dm   = make_matrix(em['doc2vec_PV_DMO'])
w2v_cbow = make_matrix(em['w2v_cbow_avg'])
w2v_sg   = make_matrix(em['w2v_skipgram_avg'])
ft_cbow  = make_matrix(em['ft_cbow_avg'])
ft_sg    = make_matrix(em['ft_skip_gram_avg'])
bert     = make_matrix(em['bert'])

In [14]:
print(tfidf.shape)
print(tfidf_w2v_cbow.shape)
print(tfidf_w2v_sg.shape)
print(tfidf_ft_cbow.shape)
print(tfidf_ft_sg.shape)
print(d2v_dvow.shape)
print(d2v_dm.shape)
print(w2v_cbow.shape)
print(w2v_sg.shape)
print(ft_cbow.shape)
print(ft_sg.shape)
print(bert.shape)

(28569, 9798)
(28569, 300)
(28569, 300)
(28569, 300)
(28569, 300)
(28569, 300)
(28569, 300)
(28569, 300)
(28569, 300)
(28569, 300)
(28569, 300)
(28569, 768)


In [15]:
print(tfidf.isnull().sum().max())
print(tfidf_w2v_cbow.isnull().sum().max())
print(tfidf_w2v_sg.isnull().sum().max())
print(tfidf_ft_cbow.isnull().sum().max())
print(tfidf_ft_sg.isnull().sum().max())
print(d2v_dvow.isnull().sum().max())
print(d2v_dm.isnull().sum().max())
print(w2v_cbow.isnull().sum().max())
print(w2v_sg.isnull().sum().max())
print(ft_cbow.isnull().sum().max())
print(ft_sg.isnull().sum().max())
print(bert.isnull().sum().max())

0
21
21
8
21
0
0
8
8
0
0
0


In [16]:
vocab_size = 9798

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16, input_shape=(None,)))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

W0627 08:32:20.279562 139924853868416 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0627 08:32:20.317492 139924853868416 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          156768    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 157,057
Trainable params: 157,057
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='binary_crossentropy',
              metrics=['acc'])

W0627 08:33:05.972093 139924853868416 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [54]:
from sklearn.model_selection import train_test_split

y_t = df['L1']
X_t = tfidf
#X_t = tfidf_w2v_cbow
# X_t = tfidf_w2v_sg
# X_t = tfidf_ft_cbow
# X_t = tfidf_ft_sg
# X_t = d2v_dvow
# X_t = d2v_dm
# X_t = w2v_cbow
# X_t = w2v_sg
# X_t = ft_cbow
# X_t = ft_sg
# X_t = bert

X_t.replace([np.nan, np.inf, -np.inf], 0, inplace=True)

print("shape of Y :"+str(y_t.shape))
print("shape of X :"+str(X_t.shape))

shape of Y :(28569,)
shape of X :(28569, 9798)


In [0]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df['L1'])
y_t = le.transform(y_t) 

In [56]:
train_data,test_data,train_labels,test_labels = train_test_split(X_t, y_t, test_size=.20, random_state=42)
print("shape of X Train :"+str(train_data.shape))
print("shape of X Test :"+str(test_data.shape))
print("shape of Y Train :"+str(train_labels.shape))
print("shape of Y Test :"+str(test_labels.shape))

shape of X Train :(22855, 9798)
shape of X Test :(5714, 9798)
shape of Y Train :(22855,)
shape of Y Test :(5714,)


In [0]:
x_val = train_data[:5000]
partial_x_train = train_data[5000:]

y_val = train_labels[:5000]
partial_y_train = train_labels[5000:]

In [58]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Train on 17855 samples, validate on 5000 samples
Epoch 1/40
17855/17855 [==============================] - 12s 670us/sample - loss: 0.6346 - acc: 0.6695 - val_loss: 0.6302 - val_acc: 0.6756
Epoch 2/40
17855/17855 [==============================] - 12s 658us/sample - loss: 0.6346 - acc: 0.6695 - val_loss: 0.6304 - val_acc: 0.6756
Epoch 3/40
17855/17855 [==============================] - 12s 657us/sample - loss: 0.6346 - acc: 0.6695 - val_loss: 0.6302 - val_acc: 0.6756
Epoch 4/40
17855/17855 [==============================] - 12s 654us/sample - loss: 0.6346 - acc: 0.6695 - val_loss: 0.6302 - val_acc: 0.6756
Epoch 5/40
17855/17855 [==============================] - 12s 659us/sample - loss: 0.6346 - acc: 0.6695 - val_loss: 0.6303 - val_acc: 0.6756
Epoch 6/40
17855/17855 [==============================] - 12s 660us/sample - loss: 0.6346 - acc: 0.6695 - val_loss: 0.6301 - val_acc: 0.6756
Epoch 7/40
17855/17855 [==============================] - 12s 656us/sample - loss: 0.6347 - acc: 0.6695 -

In [59]:
results = model.evaluate(test_data, test_labels)

print(results)

5714/5714 [==============================] - 1s 131us/sample - loss: 0.6267 - acc: 0.6804
[0.6266628738921716, 0.68043405]
